# Run: rdrp1 Metagenomes

```
Lead     : ababaian
Issue    : 
Version  : v0.4.0-dev : diamond-dev branch
start    : 2021 01 15
complete : 2021 01 xx
files    : ~/serratus/notebook/210108_ab/
s3_files : s3://serratus-public/notebook/210108_ab/
output   : s3://serratus-public/out/210115_rMeta/
output   : s3://serrauts-public/out/210115_rEuk/ (accidental)
```

### Intro/Objectives

> The time between the notes relates the color to the scenes

### Metagenomic

- File: `meta_SraRunInfo.csv`

Search Term: 
```
"METAGENOMIC" NOT amplicon[All Fields] AND "platform illumina"[Properties] AND cluster_public[prop]
```

SRA Accessed: `2020/12/31`

Results: `564 278 / 566 826`

[See: 201230_sraRunInfo_Update.ipynb](./201230_sraRunInfo_Update.ipynb)


In [ ]:
## Fire up EC2 Instance
#sudo yum install -y docker
#sudo yum install -y git
#sudo service docker start

## Download latest serratus repo
#git clone -b diamond-dev https://github.com/ababaian/serratus.git; cd serratus/containers

## If you want to upload containers to your repository, include this.
#export DOCKERHUB_USER='serratusbio' # optional
#sudo docker login # optional
#
## Build all containers and upload them docker hub repo (if available)
#./build_containers.sh

### Initialize local workspace

In [1]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS

# Create local run directory
WORK="$SERRATUS/notebook/210111_ab"
mkdir -p $WORK; cd $WORK

# S3 notebook path
S3_WORK='s3://serratus-public/notebook/210111_ab'

# date and version
date
git rev-parse HEAD # commit version

Fri Jan 15 11:29:08 PST 2021
2ddf24166fdded0bc89a96b2541bcbc7442f98f9


In [20]:
cd $WORK

# Copy from pmito5 pilot test
ln -s ../201230_sra/meta_SraRunInfo.csv ./
rm -f completed.sra; ln -s ../201230_sra/completed.sra ./

md5sum meta*

Connection to ec2-3-228-126-63.compute-1.amazonaws.com closed by remote host.
7ffe7e78bc7df55bca73eaac20a026e2  meta_SraRunInfo.csv


In [3]:
# Chop up file for batch-runs
head -n 275000 meta_SraRunInfo.csv > meta_SraRunInfo_1.csv

head -n 1 meta_SraRunInfo.csv       > meta_SraRunInfo_2.csv
tail -n+275000 meta_SraRunInfo.csv >> meta_SraRunInfo_2.csv

In [ ]:
remove_done () {
  COMPLETE=$1
  SRA=$2
  
  # Sort unique SRA in the input RunInfo file
  cut -f1 -d',' $SRA \
    | sort - \
    > inputsra.tmp
  
  # Ensure completed SRA list is sorted
  sort $COMPLETE > complete.sort.tmp
  
  # Supress SRA-accessions only in file 1 or file 1+2
  # Show SRA unique to file 2
  # parse into a regex expression to find matching lines
  comm -1 -3 complete.sort.tmp inputsra.tmp \
    | sed 's/^$\n//g' - \
    | sed 's/^/^/g' - \
    | sed 's/$/,/g' \
    > incomplete.in.$SRA
    
  #cat head -n1 $SRA > todo.$SRA  
  #while read -r line; do 
  #  grep $line $SRA >> todo.$SRA
  #done < incomplete.in.$SRA
  
  rm *.tmp
  # Still crashing due to high memory
  # but will give a list of "undone" accessions
}

#remove_done completed.sra meta_SraRunInfo_1.csv # 
#remove_done completed.sra meta_SraRunInfo_2.csv #

### Terraform Initialize

In [ ]:
## UPDATE serratus/terraform/main/main.tf
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

In [4]:
# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..a69fe44 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,68 +89,73 @@ re

     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corres

module.align.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-align]
module.download.module.iam_role.aws_iam_role_policy.cloudwatch: Creating...
module.merge.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-merge-20210115081041321700000005]
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-merge:CloudWatchLogsCreate-serratus-merge]
module.download.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creating...
module.merge.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 2s [id=profile-serratus-merge]
module.scheduler.aws_ecs_task_definition.scheduler: Creating...
module.align.aws_iam_role_policy.ec2Describe: Creating...
module.scheduler.aws_iam_role_policy.scheduler: Creation 

module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-0951527450b09c531]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-0ef22eb899b6aa527]
local_file.hosts: Creating...
local_file.create_tunnel: Creating...
local_file.create_tunnel: Creation complete after 0s [id=3590fa0b8b160bfe1298a0ce65e36420a1e70bef]
module.merge.aws_launch_configuration.worker: Creating...
module.align.aws_launch_configuration.worker: Creating...
local_file.hosts: Creation complete after 0s [id=24fbd752afbcee0309d4179e029807ab27cf1360]
module.download.aws_launch_configuration.worker: Creating...
module.download.aws_l

In [2]:
# BATCH 2

# For rapid batching; copy out serratus folder
# PROTEIN / DNA MUST BE SET IN CONFIG FILE
# main/main.tf LINE 166
#   options            = "-k ${module.work_bucket.name} -a bowtie2"
#   options            = "-k ${module.work_bucket.name} -a diamond"
# Set output location
# main/main.tf LINE 187 = "-b s3://serratus-public/out/yymmdd_id"

TF=$SERRATUS/terraform/main
cd $TF
git diff main.tf
terraform init

# Launch Terraform Cluster
# Initialize the serratus cluster with minimal nodes
terraform apply -auto-approve

diff --git a/terraform/main/main.tf b/terraform/main/main.tf
index de2d00d..6253c85 100644
--- a/terraform/main/main.tf
+++ b/terraform/main/main.tf
@@ -12,14 +12,14 @@ variable "aws_region" {
 }
 
 variable "dl_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of downloader nodes (ASG)"
 }
 
 variable "align_size" {
-  type    = number
-  default = 0
+  type        = number
+  default     = 0
   description = "Default number of aligner nodes (ASG)"
 }
 
@@ -38,14 +38,22 @@ variable "dockerhub_account" {
 }
 
 variable "scheduler_port" {
-  type  = number
+  type    = number
   default = 8000
 }
 
+variable "output_bucket" {
+  type = string
+}
+
+//variable "metrics_ip" {
+//  type = string
+//}
+
 // PROVIDER/AWS ##############################
 provider "aws" {
-  version     = "~> 2.49"
-  region      = var.aws_region
+  version = "~> 2.49"
+  region  = var.aws_region
 }
 
 provider "local" {
@@ -81,68 +89,73 @@ re

     #!/bin/bash
     set -eu
     ssh -Nf -L 3000:localhost:3000 -L 9090:localhost:9090 ec2-user@${module.monitoring.public_dns}
@@ -208,9 +216,9 @@ resource "local_file" "create_tunnel" {
 }
 
 resource "local_file" "upload_sra" {
-  filename = "${path.module}/uploadSRA.sh"
+  filename        = "${path.module}/uploadSRA.sh"
   file_permission = 0777
-  content = <<-EOF
+  content         = <<-EOF
     #!/bin/bash
     # =====================================
     # Serratus - uploadSRA.sh
@@ -289,7 +297,6 @@ resource "local_file" "upload_sra" {
   EOF
 }
 
-
 // OUTPUT ##############################
 output "help" {
   value = <<-EOF
Initializing modules...

Initializing the backend...

Initializing provider plugins...

The following providers do not have any version constraints in configuration,
so the latest version was installed.

To prevent automatic upgrades to new major versions that may contain breaking
changes, it is recommended to add version = "..." constraints to the
corres

module.merge.aws_iam_role_policy.ec2Terminate: Creating...
module.merge.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-merge:CloudWatchLogsCreate-serratus-merge]
module.merge.aws_iam_role_policy.AdjustAutoScaling: Creating...
module.download.module.iam_role.aws_iam_instance_profile.profile: Creation complete after 1s [id=profile-serratus-dl]
module.merge.aws_iam_role_policy.ec2Describe: Creating...
module.align.module.iam_role.aws_iam_role_policy_attachment.attachment["arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"]: Creation complete after 1s [id=SerratusIamRole-serratus-align-20210115210604298600000005]
module.scheduler.module.ecs_cluster.aws_iam_instance_profile.p: Creating...
module.align.module.iam_role.aws_iam_role_policy.cloudwatch: Creation complete after 1s [id=SerratusIamRole-serratus-align:CloudWatchLogsCreate-serratus-align]
module.scheduler.module.ecs_cluster.aws_iam_role_policy_attachment.instance_attachment: Creat

module.scheduler.aws_eip.sch: Creating...
module.monitoring.aws_ecs_task_definition.monitor: Creating...
module.monitoring.aws_eip.monitor: Creation complete after 3s [id=eipalloc-0da77d00a0cf55794]
module.monitoring.aws_ecs_task_definition.monitor: Creation complete after 1s [id=monitor]
module.monitoring.aws_ecs_service.monitor: Creating...
module.monitoring.aws_ecs_service.monitor: Creation complete after 1s [id=arn:aws:ecs:us-east-1:797308887321:service/serratus-monitor/serratus-monitor]
module.scheduler.aws_eip.sch: Creation complete after 3s [id=eipalloc-04dee6fdbfdd2435e]
local_file.hosts: Creating...
local_file.create_tunnel: Creating...
local_file.create_tunnel: Creation complete after 0s [id=33085f4606bec7ade3cbb1072c06993e737ee4f8]
module.download.aws_launch_configuration.worker: Creating...
module.merge.aws_launch_configuration.worker: Creating...
local_file.hosts: Creation complete after 0s [id=f2df0b32b267518fca20e4a0322d4569f2a14685]
module.align.aws_launch_configuration

Serratus Config backup 
```
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 25,
  "ALIGN_SCALING_CONSTANT": 0.0215,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 1000,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 10,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 550,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 10,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}
```

In [5]:
cd $TF

# Open SSH tunnels to the monitor
./create_tunnels.sh

# If you get an error on port
# run:
# ps aux | grep ssh
# sudo kill <PID of SSH>

Tunnels created:
    localhost:3000 = grafana
    localhost:9090 = prometheus
    localhost:5432 = postgres
    localhost:8000 = scheduler


In [7]:
cd $WORK
BATCH='meta_SraRunInfo_1.csv'
wc -l $WORK/$BATCH

275000 /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_1.csv


In [9]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_1.csv
  date: Fri Jan 15 00:14:08 PST 2021
  wc  : 275000 /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_1.csv
  md5 : 46b17ad4d2c7a03c96a9b7049c29447e  /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_1.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
c19a0f9083457377b09b452087f82f76  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
5f372aa9bfb5dcab1348198c22b63445  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
5f4fc787c627b3345b65b90046ad89c5  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
523ddfcda00d37f8233c4a26d0032bbf  tmp.chunk03_sraRunInfo.csv
{"inserted_row

In [7]:
cd $WORK
BATCH='meta_SraRunInfo_2.csv'
wc -l $WORK/$BATCH

291828 /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_2.csv


In [9]:
# Upload SraRunInfo.csv into Serratus
cd $TF
./uploadSRA.sh $WORK/$BATCH

Loading SRARunInfo into scheduler 
  File: /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_2.csv
  date: Fri Jan 15 13:09:32 PST 2021
  wc  : 291828 /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_2.csv
  md5 : 379bfed9b76d291b19ca672f42158c82  /home/artem/serratus/notebook/210111_ab/meta_SraRunInfo_2.csv


--------------------------
tmp.chunk00
10001 tmp.chunk00_sraRunInfo.csv
6fc26d69df5f9c2af9f44af721aae99d  tmp.chunk00_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":10000}
--------------------------
tmp.chunk01
10001 tmp.chunk01_sraRunInfo.csv
b842b91945fc1e0925f5b873b27f0a31  tmp.chunk01_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":20000}
--------------------------
tmp.chunk02
10001 tmp.chunk02_sraRunInfo.csv
e05fade18865bc667f1f15be5987f154  tmp.chunk02_sraRunInfo.csv
{"inserted_rows":10000,"total_rows":30000}
--------------------------
tmp.chunk03
10001 tmp.chunk03_sraRunInfo.csv
ec4258c66516dfc93d5d1abf704e3216  tmp.chunk03_sraRunInfo.csv
{"inserted_row

## Run Serratus

In [19]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json
# Golden Ratio 1400:3200

cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -sT serratus-config.json localhost:8000/config

  Cluster Config File: 
{
  "ALIGN_ARGS": "--very-sensitive-local",
  "ALIGN_MAX_INCREASE": 40,
  "ALIGN_SCALING_CONSTANT": 0.2,
  "ALIGN_SCALING_ENABLE": true,
  "ALIGN_SCALING_MAX": 200,
  "CLEAR_INTERVAL": 999999,
  "DL_ARGS": "",
  "DL_MAX_INCREASE": 20,
  "DL_SCALING_CONSTANT": 0.1,
  "DL_SCALING_ENABLE": true,
  "DL_SCALING_MAX": 0,
  "GENOME": "rdrp1",
  "MERGE_ARGS": "protein",
  "MERGE_MAX_INCREASE": 25,
  "MERGE_SCALING_CONSTANT": 0.1,
  "MERGE_SCALING_ENABLE": true,
  "MERGE_SCALING_MAX": 200,
  "SCALING_INTERVAL": 120,
  "VIRTUAL_SCALING_INTERVAL": 35
}

{"ALIGN_ARGS":"--very-sensitive-local","ALIGN_MAX_INCREASE":40,"ALIGN_SCALING_CONSTANT":0.2,"ALIGN_SCALING_ENABLE":true,"ALIGN_SCALING_MAX":200,"CLEAR_INTERVAL":999999,"DL_ARGS":"","DL_MAX_INCREASE":20,"DL_SCALING_CONSTANT":0.1,"DL_SCALING_ENABLE":true,"DL_SCALING_MAX":0,"GENOME":"rdrp1","MERGE_ARGS":"protein","MERGE_MAX_INCREASE":25,"MERGE_SCALING_CONSTANT":0.1,"MERGE_SCALING_ENABLE":true,"MERGE_SCALING_MAX":200,"SCALING_I

### Error handling

In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

#  psql -h localhost postgres postgres -c "DELETE FROM blocks WHERE state = 'done';"

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear Done
# DELETE FROM blocks WHERE state = 'done';

# RESET STATE
# DELETE FROM blocks WHERE state = 'done';
# DELETE FROM blocks WHERE state = 'fail';
#
#
# DELETE FROM acc WHERE state = 'split_err';
# DELETE FROM acc WHERE state = 'merging';
# DELETE FROM acc WHERE state = 'merge_err';
# DELETE FROM acc WHERE state = 'split_done';


In [ ]:
# Regular Shutdown
cd $TF
terraform destroy

## Nuke Shutdown
#cd $TF
#
#aws ec2 describe-instances \
#  --filter Name=tag:Name,Values=serratus-align-instance \
#  > align_instances.json
#
#jq '.Reservations[].Instances[].InstanceId' -r align_instances.json \
#  | pv -l \
#  | xargs -n10 -P10 aws ec2 terminate-instances --instance-ids

# Notes: 



psummary: `https://serratus-public.s3.amazonaws.com/out/210108_r1p/psummary/<SRA>.psummary`

SRA site: `https://www.ncbi.nlm.nih.gov/sra/?term=<SRA>`
